In [88]:
import torch
import torch.nn as nn

import nltk
import numpy as np
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

import json
# from nltk_utils import tokenize, stem, bag_of_words
# import numpy as np

# import torch
# import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import random
import json
import torch 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out


In [90]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag


In [91]:
# from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

# print(intents)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyper paramters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size,
                        shuffle =True, num_workers=2)

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        # f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}'
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

# print(f'Final loss, loss={loss.item():.4f}')
print("Final Loss, loss{:.4f}".format(loss.item()))

epoch 100/1000, loss=0.4773.
epoch 200/1000, loss=0.0183.
epoch 300/1000, loss=0.0162.
epoch 400/1000, loss=0.0115.
epoch 500/1000, loss=0.0004.
epoch 600/1000, loss=0.0004.
epoch 700/1000, loss=0.0017.
epoch 800/1000, loss=0.0003.
epoch 900/1000, loss=0.0003.
epoch 1000/1000, loss=0.0000.
Final Loss, loss0.0000


In [92]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hiddent_size": hidden_size,
    "all_words":all_words,
    "tags": tags
}
FILE = "data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

Traning complete. file saved to data.pth


In [93]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

with open('intents.json','r') as f:
  intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hiddent_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

cpu


NeuralNet(
  (l1): Linear(in_features=76, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=11, bias=True)
  (relu): ReLU()
)

In [95]:
bot_name = "Taru"
print("Hi! Welcome to cakegift.in. /n How May I Help You Today? /n Let's Chat! Type 'quit' to exit")
print("Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 7726996443.")
while True:
  sentence = input('You: ')
  for word in sentence.split():
   if word.isdigit():
     print("{} Yes, Delivery available!".format(bot_name))
     sentence = input('You: ')
     break

  if sentence =='quit':
    break
  
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]


  if prob.item() >0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(bot_name, ":",random.choice(intent['responses']))
        # print("{}:{}".format(bot_name,random.choice(intent['responses'])))
      
  else:
    print("{} I do not understand...contact on WhatsApp".format(bot_name))

Hi! Welcome to cakegift.in. /n How May I Help You Today? /n Let's Chat! Type 'quit' to exit
Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 7726996443.
You: i want to know about my order
Taru : To check the delivery status-related query please send a Whatsapp message at 9829006028, Our team reply there and resolved your concern only on the WhatsApp. Thanks
You: i want my refund
Taru I do not understand...
You: i want to cancel my order
Taru : For cancel your sent mail on order@cakegift.in
You: quit


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/taruntiwarihp/dataSets/master/CakeGift/chatHistory.csv")

In [ ]:
df.columns

Index(['S.No', 'Visitor ID', 'Name', 'Email Address', 'Phone Number', 'Status',
       'Country', 'Department', 'Embed', 'Attender', 'Question', 'Date',
       'In Time', 'Pick up Time', 'Chat Duration'],
      dtype='object')

In [ ]:
df.head()

,S.No,Visitor ID,Name,Email Address,Phone Number,Status,Country,Department,Embed,Attender,Question,Date,In Time,Pick up Time,Chat Duration
0,1,#19330,Mayank Shekhar,vu3pef@gmail.com,NaN,Missed,India,Cakegift.in,Cakegift.in,NaN,Order number 115735 not received by midnight,7/3/2021,13:35,NaN,NaN
1,2,#19329,Visitor 390961,NaN,NaN,Attended Online,India,Cakegift.in,Cakegift.in,sandeep choudhary,400070,7/3/2021,10:59,10:59,1 Min 42 Secs
2,3,#19328,Visitor 958391,NaN,NaN,Attended Online,India,Cakegift.in,Cakegift.in,sandeep choudhary,Gonda 271001,7/3/2021,10:54,10:54,8 Mins 21 Secs
3,4,#19327,Trushna jamgade,trushnajamgade94@gmail.com,7263976693,Attended Online,India,Cakegift.in,Cakegift.in,sandeep choudhary,I want order today at11pm,7/3/2021,10:46,10:46,40 Secs
4,5,#19326,vu3pef,NaN,NaN,Attended Online,India,Cakegift.in,Cakegift.in,sandeep choudhary,842002,7/3/2021,10:40,10:40,6 Mins 31 Secs


In [ ]:
df['Question'].value_counts()[:20]

Hi                   1029
Hello                 376
hi                    247
Hii                    83
hello                  81
Ok                     50
Hey                    42
Hlo                    36
Where is my order      29
Hai                    24
HI                     22
Hiii                   20
248001                 18
764001                 17
534447                 17
628501                 17
401107                 15
Hi Sandeep             15
Hi Palak               14
506101                 12
Name: Question, dtype: int64

In [ ]:
df['Question'][6]

'842002'

In [ ]:
lis = df['Question'].value_counts()

In [ ]:
pf['Question'].value_counts()[520:570]

680503                                   2
415604                                   2
614602                                   2
641001                                   2
627813                                   2
Is my order delivered or not             2
400097                                   2
764004                                   2
How much time it will take to deliver    2
Ho                                       2
523357                                   2
723121                                   2
560090                                   2
152107                                   2
581343                                   2
600069                                   2
248008                                   2
342026                                   2
509206                                   2
175002                                   2
680620                                   2
400009                                   2
421202                                   2
i have orde

In [ ]:
Can the rainbow cake be customised?

In [ ]:
pf = pf[pf.Question != 'Hiii']

In [ ]:
I have ordered a cake for today. Please let me know the delivery status.order number 115473 

I am putting msgs to u since yesterday there is no proper customer care there is no hospitality in ur website..i placed my order 112591
 yesterday for midnight delivery...paid extra for midnight delivery...no there was nothing that got deliverd..and i have no one tosk in ur website..
 no customer support and care at all...whg did u even have an option for midnight delivery...y do u even have a website wen u cant fulfill basic things..
 i placed and order and spent money to give a surprise to my husband ...now i feel foolish to GAve choosen ur website.. I need my refund ..
 i am done with your website.. 


 Can you give an update on my order 113220 which is scheduled to be delivered today between 2:30 and 4pm

 Can you please deliver the cake by 10/12/2020 11:00 am                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        2
Can I check my order status

My order from cake gift was supposed to be delivered today by 4:00pm, but it is not delivered yet. I want to know the status for my order no. 109272 

Hi are ur services available for the below mentioned address

Hi there\nI have placed an order an month ago- 108842 for yesterday& 39;s delviery. It& 39;s still not been delivered. May I know what&#39;s happening?\nLooking for earliest reply.\n\nThanks\nSahithi Jadav

In [ ]:
lis[:30]

Hi                   1029
Hello                 376
hi                    247
Hii                    83
hello                  81
Ok                     50
Hey                    42
Hlo                    36
Where is my order      29
Hai                    24
HI                     22
Hiii                   20
248001                 18
764001                 17
534447                 17
628501                 17
401107                 15
Hi Sandeep             15
Hi Palak               14
506101                 12
606202                 12
635601                 12
521235                 12
hey                    12
690101                 12
632602                 11
Hi sir                 10
Hi Pooja               10
Hi,                    10
628502                  9
Name: Question, dtype: int64